In [ ]:
import pandas as pd
df_comb_rm = pd.read_csv('datasets/base_combustiveis_precos.csv', low_memory=False, decimal=',')

In [ ]:
df_comb_rm.rename(columns={'Estado - Sigla': 'UF', 'Valor de Venda': 'Valor_Venda', 'Data da Coleta': 'Data'}, inplace = True)

df_rms = pd.read_csv('datasets/rms_uf_resumida_rev02.csv')

df_comb_rm = df_comb_rm[df_comb_rm[['UF', 'Municipio']].apply(tuple, axis=1).isin(df_rms[['UF','Municipio']].apply(tuple, axis=1))]
df_comb_rm = pd.merge(df_comb_rm, df_rms[['Municipio', 'UF', 'RM']], on=['Municipio', 'UF'], how='left')

df_comb_rm['Data'] = pd.to_datetime(df_comb_rm['Data'])

In [ ]:
df_comb_rm

In [ ]:
df_comb_rm.dtypes

In [ ]:
df_comb_rm['Valor_Venda'] = df_comb_rm['Valor_Venda'].apply(lambda x: x.replace(',', '.'))
df_comb_rm['Valor_Venda'] = pd.to_numeric(df_comb_rm['Valor_Venda'])
df_comb_rm['Ano'] = pd.DatetimeIndex(df_comb_rm['Data']).year

In [ ]:
df_comb_rm

In [ ]:
df_comb_rm.dtypes

In [ ]:
df_comb_rm_g = df_comb_rm.groupby(['RM', 'Ano', 'Produto',])['Valor_Venda'].mean()
df_comb_rm_g = df_comb_rm_g.reset_index()

df_comb_rm_g

#df_comb_rm_g.to_csv('datasets/base_combustiveis_precos_rm.csv')

In [ ]:
df_comb_rm_rel = df_comb_rm_g[((df_comb_rm_g['Produto'] == 'GASOLINA') | (df_comb_rm_g['Produto'] == 'ETANOL'))]
df_comb_rm_rel

#preco_gasolina = df_preco_comb_rm[((df_preco_comb_rm['Produto'] == 'GASOLINA') & (df_preco_comb_rm['Ano'] == ano) & (df_preco_comb_rm['RM'] == df['RM']))]['Preco_Venda']
#preco_alcool = df_preco_comb_rm[((df_preco_comb_rm['Produto'] == 'ALCOOL') & (df_preco_comb_rm['Ano'] == ano) & (df_preco_comb_rm['RM'] == df['RM']))]['Preco_Venda']

In [ ]:
df_comb_rm_rel = df_comb_rm_rel.pivot(index=['RM', 'Ano'], columns='Produto', values='Valor_Venda')
df_comb_rm_rel = df_comb_rm_rel.reset_index()
df_comb_rm_rel

In [ ]:
df_comb_rm_rel['Relacao'] = (1 / (1 + 0.001685*((6567.5754)**(df_comb_rm_rel['ETANOL']/df_comb_rm_rel['GASOLINA']))))
df_comb_rm_rel

In [ ]:
df_comb_rm_rel.to_csv('datasets/base_combustiveis_precos_rel_rm.csv')